In [2]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder

2023-04-11 09:03:50.685617: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 09:03:50.968931: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


This is the data set used by Gorman and Sejnowski in their study of the classification of sonar signals using a neural network: Gorman, R. P., and Sejnowski, T. J. (1988);  “Analysis of Hidden Units in a Layered Network Trained to Classify Sonar Targets” in Neural Networks, Vol. 1, pp. 75-89.
The task is to train a network to discriminate between sonar signals bounced off a metal cylinder and those bounced off a roughly cylindrical rock.
The original files can be found at the adress: http://archive.ics.uci.edu/ml/datasets/connectionist+bench+(sonar,+mines+vs.+rocks)

The file “sonar.mines” contains 111 patterns obtained by bouncing sonar signals off a metal cylinder at various angles and under various conditions. The file “sonar.rocks” contains 97 patterns obtained from rocks under similar conditions. The transmitted sonar signal is a frequency-modulated chirp, rising in frequency. The data set contains signals obtained from a variety of different aspect angles, spanning 90 degrees for the cylinder and 180 degrees for the rock.

Each pattern is a set of 60 numbers in the range 0.0 to 1.0. Each number represents the energy within a particular frequency band, integrated over a certain period of time. The integration aperture for higher frequencies occur later in time, since these frequencies are transmitted later during the chirp.

The label associated with each record contains the letter “R” if the object is a rock and “M” if it is a mine (metal cylinder). The numbers in the labels are in increasing order of aspect angle, but they do not encode the angle directly.

In [4]:
# load dataset
# the file sonar.all-data of the UCI repository is renamed "sonar.csv"
dataframe = pd.read_csv("sonar.csv", header=None)
data = dataframe.values

## Separation between training and testing

In [5]:
print(data.shape)

(208, 61)


In [6]:
from sklearn.model_selection import train_test_split

First, we write a function that gives the repartition between the two labels in a given dataset.

In [7]:
def repartition(tab,label):
    nR=0; nM=0
    nb = len(tab[:,60])
    print(nb)
    for i in range(nb):
        if(tab[i,60]=="R"):
            nR+=1
        else:
            nM+=1
    print("R ", label, ": ", 100*nR/nb)
    print("M ",label, ": ", 100*nM/nb)
repartition(data,"total")

208
R  total :  46.63461538461539
M  total :  53.36538461538461


The goal is to split in 50% training and testing in order that the training and the testing dataset have almost the same label distribution. We try different random_state parameter for the built-in function train_test_split until getting something reasonable.

In [8]:
train_set,test_set = train_test_split(data, test_size=0.5, random_state=7)

In [9]:
repartition(data,"total")
repartition(train_set,"train")
repartition(test_set,"test")

208
R  total :  46.63461538461539
M  total :  53.36538461538461
104
R  train :  48.07692307692308
M  train :  51.92307692307692
104
R  test :  45.19230769230769
M  test :  54.80769230769231


In [11]:
# split into input (X) and output (Y) variables
X_train = train_set[:,0:60].astype(float)
Y_train = train_set[:,60]
X_test = test_set[:,0:60].astype(float)
Y_test = test_set[:,60]

# Encode the output 

In the original database the output are labeled by letter "R" or "M". Here, we assign number 0 or 1 to each label(one-hot encoding).

In [12]:
encoder = LabelEncoder()
encoder.fit(Y_train)
encoded_Y_train = encoder.transform(Y_train)
encoded_Y_test = encoder.transform(Y_test)

# Normalization of the inputs

We do a standard normalization (mean 0 and standard deviation 1) in order to have the same scale for the inputs.

In [13]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [14]:
transformer = StandardScaler().fit(X_train)

In [15]:
X_train_prepared = transformer.transform(X_train)
X_test_prepared = transformer.transform(X_test)

# Record the data

In [16]:
import numpy as np

We record the preprocessing data in csv files that we will use in the C/C++ code to train the network.

In [14]:
np.savetxt('sonar_inputs_train.csv', X_train_prepared, delimiter=",")
np.savetxt('sonar_outputs_train.csv', encoded_Y_train, delimiter=",")
np.savetxt('sonar_inputs_test.csv', X_test_prepared, delimiter=",")
np.savetxt('sonar_outputs_test.csv', encoded_Y_test, delimiter=",")